In [1]:
import gensim

In [7]:
model = gensim.models.Word2Vec.load_word2vec_format('./gn/GoogleNews-vectors-negative300.bin', binary=True)  # C binary format

In [14]:
stoplist = ['.', ',', ':', '"', '?', '!', '<', '>', '(', ')', '@', '#', '$', '^', '*', '/', '\n', ' ', '-', '_', 'a', 'the']
def tokenize(string, tokens):
    """
    Разбивает входную строку на токены
    :type string: str
    :param string: строка, которая будет разбита на токены
    :type tokens: list
    :param tokens: список токенов
    """
    current_word = str('')
    for char in string:
        if char.isalpha() and char not in stoplist:
            current_word += char
            continue
        else:
            if len(current_word) > 0:
                if current_word.lower() not in tokens and current_word not in stoplist:
                    tokens.append(current_word.lower())
            if char.isalpha() and char not in stoplist:
                current_word = char
            else:
                current_word = str('')

    if len(current_word) > 0:
        if current_word.lower() not in tokens and current_word not in stoplist:
            tokens.append(current_word.lower())

In [15]:
Paragraph1 = 'A single Kingdom of Great Britain resulted from the Union of Scotland and England (which already comprised the present-day countries of England and Wales) in 1707'
par1 = []
tokenize(Paragraph1, par1)

In [16]:
Paragraph2 = 'Great Britain refers geographically to the island of Great Britain, politically to England, Scotland and Wales in combination.[27] However, it is sometimes used loosely to refer to the whole of the United Kingdom'
par2 = []
tokenize(Paragraph2, par2)

In [19]:
def paragraph_vec(paragraph):
    """
    Подсчитывает сумму векторов слов абзаца
    :type paragraph: list of strings
    :param paragraph: список всех слов абзаца
    :return: list 300-vector
    """
    par_vec = []
    for x in xrange(300):
        res = 0.0
        error = 0.0
        for word in paragraph:
            if word in model:
                y = model[word][x] - error
                t = res + y
                error = (t - res) - y
                res = t
        par_vec.append(res)
    return par_vec

In [20]:
par1_vec = paragraph_vec(par1)
par2_vec = paragraph_vec(par2)

In [23]:
def cosine(vec1, vec2):
    len1 = norm(vec1)
    len2 = norm(vec2)
    if len1 == 0 or len2 == 0:
        return 0
    else:
        cos = scalar_prod(vec1, vec2)/(len1*len2)
    return cos

In [24]:
import numpy as np
def scalar_prod(vec1, vec2):
    """
    Подсчитывает скалярное произведение двух векторов
    :type vec1: list
    :param vec1: 300-вектор первого слова
    :type vec2: list
    :param vec2: 300-вектор второго слова
    :return: float
    """
    return np.dot(vec1, vec2)

In [26]:
from math import sqrt
def norm(vec):
    return sqrt(scalar_prod(vec, vec))

In [27]:
cos = cosine(par1_vec, par2_vec)
print cos

0.829489248835


In [28]:
def max_cosines(text1, text2):
    """
    Подсчитывает максимальное косинусное расстоение между словом первого абзаца и всеми словами второго.
    Затем усредняет все эти значения.
    :type text1: list
    :param text1: список всех слов первого текста
    :type text2: list
    :param text2: список всех слов второго текста
    :return: float
    """
    max_cosines = [max([cosine(model[w1], model[w2]) for w2 in text2 if w2 in model]) for w1 in text1 if w1 in model] 
    av = sum(max_cosines)/len(max_cosines)
    return av

In [30]:
dataset = open("gutenberg_task.txt", 'r')

In [31]:
def split_line(line):
    """
    Разделяет текст по знакам табуляции и сохраняет в список
    :type line: string
    :param line: текст
    :return: list
    """
    sents = []
    first_sym_num = 0
    last_sym_num = 0
    for symbol in line:
        last_sym_num += 1
        if symbol == '\t':
            word = line[first_sym_num:last_sym_num]
            sents.append(word)
            first_sym_num = last_sym_num
    word = line[first_sym_num:]
    sents.append(word)
    return sents

In [32]:
text = []
for line in dataset:
    text.append(split_line(line))

In [ ]:
res1 = []
res2 = []
i = 0
for paragraph in text:
    if i % 50 == 0:
        print i
    sent1 = paragraph[0]
    sent2 = paragraph[1]
    sent3 = paragraph[2]
    sent1_list = []
    sent2_list = []
    sent3_list = []
    tokenize(sent1, sent1_list)
    tokenize(sent2, sent2_list)
    tokenize(sent3, sent3_list)
    sent1_vec = paragraph_vec(sent1_list)
    sent2_vec = paragraph_vec(sent2_list)
    sent3_vec = paragraph_vec(sent3_list)
    res1_par = [] 
    res2_par = []
    res1_par.append(cosine(sent1_vec, sent2_vec))
    res1_par.append(cosine(sent1_vec, sent3_vec))
    res1.append(res1_par)
    res2_par.append(max_cosines(sent1_list, sent2_list))
    res2_par.append(max_cosines(sent1_list, sent3_list))
    res2.append(res2_par)
    i += 1

In [34]:
l1 = np.array(map(lambda x: int(x[0]>x[1]), res1))
l2 = np.array(map(lambda x: int(x[0]>x[1]), res2))

In [35]:
#Доверительные интервалы уровня доверия 0,95. нужная квантиль - 1,96
m1 = l1.mean()
m2 = l2.mean()
d1 = 1.96*sqrt(m1*(1-m1)/len(l1))
d2 = 1.96*sqrt(m2*(1-m2)/len(l2))
print "Точность первого алгоритма:", m1
print "Точность второго алгоритма:", m2
print "Доверительный интервал первого алгоритма:", (m1-d1, m1+d1)
print "Доверительный интервал второго алгоритма:", (m2-d2, m2+d2)

Точность первого алгоритма: 0.6694
Точность второго алгоритма: 0.6091
Доверительный интервал первого алгоритма: (0.66017958751777339, 0.6786204124822266)
Доверительный интервал второго алгоритма: (0.59953614008308354, 0.61866385991691641)
